In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os
import time

import numpy as np
import cv2
import scipy
from pydub import AudioSegment
from sklearn.utils import shuffle
import queue
import os

import numpy as np
import librosa
from pathlib import Path
import pickle
import cv2
import pandas as pd

import numpy as np
import os
from scipy import  misc
from keras.models import model_from_json
import keras

import tensorflow as tf


print ("HANDLING IMPORTS...")
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function
import sys
import os
import time
import operator
import math
import queue
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
#import Queue as queue
import csv
import copy
import random
from scipy import interpolate
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
import itertools
import pickle
import numpy
import matplotlib.pyplot as plt
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD,Adam
#from keras.optimizers.schedules import ExponentialDecay
import keras
from keras.layers import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.regularizers import l1,l2
from keras.constraints import max_norm
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
import cv2
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from keras import initializers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.engine.base_layer import Layer
from keras import backend as K
from keras.backend import clear_session
from keras import backend
# force channels-last ordering
backend.set_image_data_format('channels_first')
print(backend.image_data_format())


HANDLING IMPORTS...
channels_first


In [2]:
os.listdir('../input/birdsong-recognition')

['train.csv',
 'example_test_audio_metadata.csv',
 'train_audio',
 'test.csv',
 'example_test_audio_summary.csv',
 'example_test_audio',
 'sample_submission.csv']

In [3]:
#labels as Pickle file
classifier_f = open("../input/16thaugust-newactivation/int_to_word_spec_new_spec_bandpass_512_128.pickle", "rb")

int_to_word_out = pickle.load(classifier_f)
classifier_f.close()

In [4]:
###############################################
def getRandomState():

    RANDOM_SEED = 1337
    RANDOM = np.random.RandomState(RANDOM_SEED)

    return RANDOM
######################## CONFIG #########################
RANDOM = getRandomState()


SAMPLE_RATE= 44100
SPEC_LENGTH= 5.0
SPEC_OVERLAP= 0.0
SPEC_MINLEN= 1.0
SPEC_FMIN= 500
SPEC_FMAX= 22000
SPEC_TYPE='melspec'

SPEC_SIGNAL_THRESHOLD= 0.0001
IM_DIM = 1
magnitude_scale = 'nonlinear'
bandpass=True 
IM_SIZE = (512, 128)
BATCH_SIZE=16
NUM_CLASSES=264

In [5]:
import os
import time

import numpy as np
import cv2

from sklearn.utils import shuffle

import os

import numpy as np
import librosa

import cv2
#import audio
################################################
import scipy
from scipy import signal
import cv2
RANDOM_SEED = 1337
RANDOM = np.random.RandomState(RANDOM_SEED)
CACHE = {}
def openAudioFile(path, sample_rate=44100, as_mono=True, mean_substract=False):
    
    # Open file with librosa (uses ffmpeg or libav)
    sig, rate = librosa.load(path, sr=sample_rate, mono=as_mono,res_type="kaiser_fast")

    # Noise reduction?
    if mean_substract:
        sig -= sig.mean()

    return sig, rate

def noise(sig, shape, amount=None):

    # Random noise intensity
    if amount == None:
        amount = RANDOM.uniform(0.1, 0.9)

    # Create Gaussian noise
    noise = RANDOM.normal(min(sig) * amount, max(sig) * amount, shape)

    return noise.astype('float32')

def signal2noise(spec):

    # Get working copy
    spec = spec.copy().astype('float32')

    # Calculate median for columns and rows
    col_median = np.median(spec, axis=0, keepdims=True)
    row_median = np.median(spec, axis=1, keepdims=True)

    # Binary threshold
    spec[spec < row_median * 1.25] = 0.0
    spec[spec < col_median * 1.15] = 0.0
    spec[spec > 0] = 1.0

    # Median blur
    spec = cv2.medianBlur(spec, 3)

    # Morphology
    spec = cv2.morphologyEx(spec, cv2.MORPH_CLOSE, np.ones((3, 3), np.float32))

    # Sum of all values
    spec_sum = spec.sum()

    # Signal to noise ratio (higher is better)
    try:
        s2n = spec_sum / (spec.shape[0] * spec.shape[1] * spec.shape[2])
    except:
        s2n = spec_sum / (spec.shape[0] * spec.shape[1])

    return s2n

def splitSignal(sig, rate, seconds, overlap, minlen):

    # Split signal with overlap
    sig_splits = []
    for i in range(0, len(sig), int((seconds - overlap) * rate)):
        split = sig[i:i + int(seconds * rate)]

        # End of signal?
        if len(split) < int(minlen * rate):
            break
        
        # Signal chunk too short?
        if len(split) < int(rate * seconds):
            split = np.hstack((split, noise(split, (int(rate * seconds) - len(split)), 0.5)))
        
        sig_splits.append(split)

    return sig_splits


def buildBandpassFilter(rate, fmin, fmax, order=4):

    global CACHE

    fname = 'bandpass_' + str(rate) + '_' + str(fmin) + '_' + str(fmax)
    if not fname in CACHE:
        wn = np.array([fmin, fmax]) / (rate / 2.0)
        filter_sos = scipy.signal.butter(order, wn, btype='bandpass', output='sos')

        # Save to cache
        CACHE[fname] = filter_sos

    return CACHE[fname]

def applyBandpassFilter(sig, rate, fmin, fmax):

    # Build filter or load from cache
    filter_sos = buildBandpassFilter(rate, fmin, fmax)

    return scipy.signal.sosfiltfilt(filter_sos, sig)

def pcen(spec, rate, hop_length, gain=0.8, bias=10, power=0.25, t=0.060, eps=1e-6):
    s = 1 - np.exp(- float(hop_length) / (t * rate))
    M = scipy.signal.lfilter([s], [1, s - 1], spec)
    smooth = (eps + M)**(-gain)
    return (spec * smooth + bias)**power - bias**power

def getMelFilterbank(num_banks, fmin, fmax, f_vec, dtype=np.float32):
    '''
    An arguably better version of librosa's melfilterbanks wherein issues with "hard snapping" are avoided. Works with
    an existing vector of frequency bins, as returned from signal.spectrogram(), instead of recalculating them and
    flooring down the bin indices.
    '''

    global CACHE

    # Filterbank already in cache?
    fname = 'mel_' + str(num_banks) + '_' + str(fmin) + '_' + str(fmax)
    if not fname in CACHE:
        
        # Break frequency and scaling factor
        A = 4581.0
        f_break = 1750.0

        # Convert Hz to mel
        freq_extents_mel = A * np.log10(1 + np.asarray([fmin, fmax], dtype=dtype) / f_break)

        # Compute points evenly spaced in mels
        melpoints = np.linspace(freq_extents_mel[0], freq_extents_mel[1], num_banks + 2, dtype=dtype)

        # Convert mels to Hz
        banks_ends = (f_break * (10 ** (melpoints / A) - 1))

        filterbank = np.zeros([len(f_vec), num_banks], dtype=dtype)
        for bank_idx in range(1, num_banks+1):
            # Points in the first half of the triangle
            mask = np.logical_and(f_vec >= banks_ends[bank_idx - 1], f_vec <= banks_ends[bank_idx])
            filterbank[mask, bank_idx-1] = (f_vec[mask] - banks_ends[bank_idx - 1]) / \
                (banks_ends[bank_idx] - banks_ends[bank_idx - 1])

            # Points in the second half of the triangle
            mask = np.logical_and(f_vec >= banks_ends[bank_idx], f_vec <= banks_ends[bank_idx+1])
            filterbank[mask, bank_idx-1] = (banks_ends[bank_idx + 1] - f_vec[mask]) / \
                (banks_ends[bank_idx + 1] - banks_ends[bank_idx])

        # Scale and normalize, so that all the triangles do not have same height and the gain gets adjusted appropriately.
        temp = filterbank.sum(axis=0)
        non_zero_mask = temp > 0
        filterbank[:, non_zero_mask] /= np.expand_dims(temp[non_zero_mask], 0)

        # Save to cache
        CACHE[fname] = (filterbank, banks_ends[1:-1])

    return CACHE[fname][0], CACHE[fname][1]

def spectrogram(sig, rate, shape=(128, 512), win_len=512, fmin=500, fmax=22000, frequency_scale='mel', magnitude_scale='nonlinear', bandpass=True):

    # Compute overlap
    hop_len = int(len(sig) / (shape[1] - 1)) 
    win_overlap = win_len - hop_len + 2
    #print('WIN_LEN:', win_len, 'HOP_LEN:', hop_len, 'OVERLAP:', win_overlap)

    # Adjust N_FFT?
    if frequency_scale == 'mel':
        n_fft = win_len
    else:
        n_fft = shape[1] * 2

    # Bandpass filter?
    if bandpass:
        sig = applyBandpassFilter(sig, rate, fmin, fmax)

    # Compute spectrogram
    f, t, spec = scipy.signal.spectrogram(sig,
                                          fs=rate,
                                          window=scipy.signal.windows.hann(win_len),
                                          nperseg=win_len,
                                          noverlap=win_overlap,
                                          nfft=n_fft,
                                          detrend=False,
                                          mode='magnitude')

    # Scale frequency?
    if frequency_scale == 'mel':

        # Determine the indices of where to clip the spec
        valid_f_idx_start = f.searchsorted(fmin, side='left')
        valid_f_idx_end = f.searchsorted(fmax, side='right') - 1

        # Get mel filter banks
        mel_filterbank, mel_f = getMelFilterbank(shape[0], fmin, fmax, f, dtype=spec.dtype)

        # Clip to non-zero range so that unnecessary multiplications can be avoided
        mel_filterbank = mel_filterbank[valid_f_idx_start:(valid_f_idx_end + 1), :]

        # Clip the spec representation and apply the mel filterbank.
        # Due to the nature of np.dot(), the spec needs to be transposed prior, and reverted after
        spec = np.transpose(spec[valid_f_idx_start:(valid_f_idx_end + 1), :], [1, 0])
        spec = np.dot(spec, mel_filterbank)
        spec = np.transpose(spec, [1, 0])        

    # Magnitude transformation
    if magnitude_scale == 'pcen':
        
        # Convert scale using per-channel energy normalization as proposed by Wang et al., 2017
        # We adjust the parameters for bird voice recognition based on Lostanlen, 2019
        spec = pcen(spec, rate, hop_len)
        
    elif magnitude_scale == 'log':
        
        # Convert power spec to dB scale (compute dB relative to peak power)
        spec = spec ** 2
        spec = 10.0 * np.log10(np.maximum(1e-10, spec) / np.max(spec))
        spec = np.maximum(spec, spec.max() - 100) # top_db = 100

    elif magnitude_scale == 'nonlinear':

        # Convert magnitudes using nonlinearity as proposed by Schlüter, 2018
        a = -1.2 # Higher values yield better noise suppression
        s = 1.0 / (1.0 + np.exp(-a))
        spec = spec ** s

    # Flip spectrum vertically (only for better visialization, low freq. at bottom)
    spec = spec[::-1, ...]

    # Trim to desired shape if too large
    spec = spec[:shape[0], :shape[1]]

    # Normalize values between 0 and 1
    spec -= spec.min()
    if not spec.max() == 0:
        spec /= spec.max()
    else:
        spec = np.clip(spec, 0, 1)

    return spec


def get_spec(sig, rate, shape, spec_type='linear', **kwargs):

    if spec_type.lower()== 'melspec':
        #return melspec(sig, rate, shape, **kwargs)
        return spectrogram(sig, rate, frequency_scale='mel', **kwargs)
    else:
        return stft(sig, rate, shape, **kwargs)

def signal2noise(spec):

    # Get working copy
    spec = spec.copy().astype('float32')

    # Calculate median for columns and rows
    col_median = np.median(spec, axis=0, keepdims=True)
    row_median = np.median(spec, axis=1, keepdims=True)

    # Binary threshold
    spec[spec < row_median * 1.25] = 0.0
    spec[spec < col_median * 1.15] = 0.0
    spec[spec > 0] = 1.0

    # Median blur
    spec = cv2.medianBlur(spec, 3)

    # Morphology
    spec = cv2.morphologyEx(spec, cv2.MORPH_CLOSE, np.ones((3, 3), np.float32))

    # Sum of all values
    spec_sum = spec.sum()

    # Signal to noise ratio (higher is better)
    try:
        s2n = spec_sum / (spec.shape[0] * spec.shape[1] * spec.shape[2])
    except:
        s2n = spec_sum / (spec.shape[0] * spec.shape[1])

    return s2n

def specsFromSignal(sig, rate, shape, seconds, overlap, minlen, **kwargs):

    # Split signal in consecutive chunks with overlap
    sig_splits = splitSignal(sig, rate, seconds, overlap, minlen)

    # Extract specs for every sig split
    for sig in sig_splits:

        # Get spec for signal chunk
        spec = get_spec(sig, rate, shape, **kwargs)

        yield spec

def specsFromFile(path, rate, seconds, overlap, minlen, shape, start=-1, end=-1, **kwargs):

    # Open file
    sig, rate = openAudioFile(path, rate)

    # Trim signal?
    if start > -1 and end > -1:
        sig = sig[int(start * rate):int(end * rate)]
        minlen = 0

    # Yield all specs for file
    for spec in specsFromSignal(sig, rate, shape, seconds, overlap, minlen, **kwargs):
        yield spec

In [6]:
class Mish(Layer):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X_input = Input(input_shape)
        >>> X = Mish()(X_input)
    '''

    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)
        self.supports_masking = True

    def call(self, inputs):
        return inputs * K.tanh(K.softplus(inputs))

    def get_config(self):
        base_config = super(Mish, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape




In [7]:
def mish(x):
        return keras.layers.Lambda(lambda x: x*K.tanh(K.softplus(x)))(x)

In [8]:
class LearningRateDecay:
        def plot(self, epochs, title="Learning Rate Schedule"):
                # compute the set of learning rates for each corresponding
                # epoch
                lrs = [self(i) for i in epochs]

                # the learning rate schedule
                plt.style.use("ggplot")
                plt.figure()
                plt.plot(epochs, lrs)
                plt.title(title)
                plt.xlabel("Epoch #")
                plt.ylabel("Learning Rate")


In [9]:
class StepDecay(LearningRateDecay):
        def __init__(self, initAlpha=0.01, factor=0.25, dropEvery=10):
                # store the base initial learning rate, drop factor, and
                # epochs to drop every
                self.initAlpha = initAlpha
                self.factor = factor
                self.dropEvery = dropEvery
        def __call__(self, epoch):
                # compute the learning rate for the current epoch
                exp = np.floor((1 + epoch) / self.dropEvery)
                alpha = self.initAlpha * (self.factor ** exp)
                # return the learning rate
                return float(alpha)
            
            


In [10]:
'''def load_model():
    
    # load json and create model
    json_file = open('16thaugust_newactivation/model_bird_spec_1staugust_spec_bandpass_aalnet_512_128_7x7filter.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("16thaugust_newactivation/model_bird_spec_31sthuly_spec_bandpass_512_128_aalnet-7x7filter-18-1.0461.h5")
    return loaded_model
'''

model = Sequential()
#Layer 1
model.add(Conv2D(32, (7,7),input_shape=(1,128, 512),activation=mish, padding='same',kernel_initializer='he_normal'))
#model.add(beta_mish())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
#Layer 2
model.add(Conv2D(64, (7,7), padding='same',activation=mish,kernel_initializer='he_normal'))
#model.add(beta_mish())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
#Layer 3    
model.add(Conv2D(128,(7,7), padding='same',activation=mish,kernel_initializer='he_normal'))
#model.add(beta_mish())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
#Layer 4
model.add(Conv2D(256,(7,7), padding='same',activation=mish, kernel_initializer='he_normal'))
#model.add(beta_mish())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
#Layer 5
model.add(Conv2D(512,(7,7), padding='same',activation=mish, kernel_initializer='he_normal'))
#model.add(beta_mish())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(256))
model.add(Mish())
model.add(BatchNormalization())
model.add(Dropout(0.5))  
model.add(Dense(128))
model.add(Mish())
#model.add(beta_mish())
model.add(BatchNormalization())
model.add(Dropout(0.2))
    
#output
model.add(Dense(264, activation='softmax'))
# Compile model
lrate = 0.0001
decay=0.0
#decay = lrate/epochs #if want to performlearning rate decay
schedule = StepDecay(initAlpha=1e-1, factor=0.25, dropEvery=15)
opt = SGD(lr=0.01, momentum=0.9, decay=decay)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) 
#return model
#model.load_weights("16thaugust_newactivation/model_bird_spec_31sthuly_spec_bandpass_512_128_aalnet-7x7filter-18-1.0461.h5")
model.summary()
#model=load_model()
model.load_weights("../input/16thaugust-newactivation/model_bird_spec_31sthuly_spec_bandpass_512_128_aalnet-7x7filter-18-1.0461.h5")




Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 128, 512)      1600      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 128, 512)      2048      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 64, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 256)       100416    
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 32, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 32, 128)      4

In [11]:
from pathlib import Path

TEST = Path("/kaggle/input/birdsong-recognition/test_audio").exists()

if TEST:
    TEST_DF_DIR = "/kaggle/input/birdsong-recognition/"
else:
    TEST_DF_DIR = "/kaggle/input/birdcall-check/"
    
test_df = pd.read_csv(f"{TEST_DF_DIR}test.csv")
test_audio = TEST_DF_DIR + "test_audio/"

In [12]:
##################################################################
# -*- coding: utf-8 -*-
#Loading images with CPU background threads during GPU forward passes saves a lot of time
#Credit: J. Schlüter (https://github.com/Lasagne/Lasagne/issues/12)
def threadedBatchGenerator(generator, num_cached=70):
    
    #import Queue
    queues = queue.Queue(maxsize=num_cached)
    sentinel = object()  # guaranteed unique reference

    #define producer (putting items into queue)
    def producer():
        for item in generator:
            queues.put(item)
        queues.put(sentinel)

    #start producer (in a background thread)
    import threading
    thread = threading.Thread(target=producer)
    thread.daemon = True
    thread.start()

    #run as consumer (read items from queue, in current thread)
    item = queues.get()
    while item is not sentinel:
        yield item
        queues.task_done()
        item = queues.get()



In [13]:
class TestDataset(keras.utils.Sequence):
    def __init__(self, df, clip, IM_SIZE = (512,128),shape=(128, 512), fmin=500,fmax=22000,seconds=5.0,overlap=0.0,minlen=1.0,win_len=512, frequency_scale='mel', magnitude_scale='nonlinear', bandpass=True,IM_DIM=1):
        self.df = df
        self.clip = clip
        self.IM_SIZE = IM_SIZE
        self.fmin=fmin
        self.shape=shape
        self.fmax=fmax
        self.frequency_scale=frequency_scale 
        self.magnitude_scale=magnitude_scale 
        self.bandpass=bandpass
        self.IM_DIM=IM_DIM
        self.win_len=win_len
        self.seconds=seconds
        self.overlap=overlap
        self.minlen=minlen
        #self.amptodb = torchaudio.transforms.AmplitudeToDB()
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx: int):
        SR = 44100
        sample = self.df.iloc[idx]
        site = sample.site
        row_id = sample.row_id
        
        if site == "site_3":
                images=[]
                noises=[]
                y = self.clip.astype(np.float32)
                sig_splits = splitSignal(y, SR, self.seconds, self.overlap, self.minlen)

    # Extract specs for every sig split
                for sig in sig_splits:
                    y=spectrogram(sig, SR, self.shape, self.win_len, self.fmin, self.fmax, self.frequency_scale, self.magnitude_scale, self.bandpass)
                    #y= melspec(sig, SR, self.shape, self.fmin, self.fmax, self.normalize, self.preemphasis)
                    s2n = signal2noise(y)
                    y = y.reshape(-1, self.IM_DIM, self.IM_SIZE[1], self.IM_SIZE[0])
                #print (y)
                    images.append(y)
                    noises.append(s2n)
                    #print (len(images))  
                #images = np.asarray(images)
            #print (len(images))
            #images.reshape(-1, self.IM_DIM, self.IM_SIZE[1], self.IM_SIZE[0])

                return images, noises, row_id, site
        else:
            #images=[]
            end_seconds = int(sample.seconds)
            start_seconds = int(end_seconds - 5)
            
            start_index = SR * start_seconds
            end_index = SR * end_seconds
            
            y = self.clip[start_index:end_index].astype(np.float32)
            #sig_splits = splitSignal(y, SR, self.seconds, self.overlap, self.minlen)
            #for sig in sig_splits:
                
            y=spectrogram(y, SR, self.shape, self.win_len, self.fmin, self.fmax, self.frequency_scale, self.magnitude_scale, self.bandpass)
            noises = signal2noise(y)
            image = y.reshape(-1, self.IM_DIM, self.IM_SIZE[1], self.IM_SIZE[0])
            #images.append(image) 
            return image, noises, row_id, site

In [14]:
%%time
prediction_dict = {}
SR=44100
for audio_id in test_df.audio_id.unique():
        y, sr = librosa.load(test_audio + (audio_id + ".mp3"),sr=SR)
        new_test_df = test_df.query(f"audio_id == '{audio_id}'").reset_index(drop=True)
        dataset = TestDataset(new_test_df, y, IM_SIZE = (512,128),shape=(128, 512), fmin=500,fmax=22000,seconds=5.0,overlap=0.0,minlen=1.0,win_len=512, frequency_scale='mel', magnitude_scale='nonlinear', bandpass=True,IM_DIM=1)
        for image, noises,  row_id, site in threadedBatchGenerator(dataset):
            #print (len(image)) 
            if site in {"site_1", "site_2"}:
                
                y=model.predict(image)
                #print (np.max(y))
                #print (noises)
                #print (int_to_word_out[np.argmax(y)])
                if np.max(y)>=0.9:
                    
                    #print (np.max(y))
                    #print ( np.max(y))
                    #print (int_to_word_out[np.argmax(y)])
                    #print (int_to_word_out[np.argmax(y)])
                    prediction_dict[row_id] = ''.join(int_to_word_out[np.argmax(y)])
                else:
                    prediction_dict[row_id]='nocall'
                    #print (prediction_dict[row_id])
            else:
                results=[]
                for s in range (len(image)):
                 #for noise in noises:  
                
                    img = image[s].reshape(-1, IM_DIM, IM_SIZE[1], IM_SIZE[0])
                    y=model.predict(img)
                    #print ( np.max(y))  
                    #print (noises[s])

                    if np.max(y)>=0.9:
                                #print ( np.max(y))
                                #print (int_to_word_out[np.argmax(y)])
                                results.append(int_to_word_out[np.argmax(y)])
                                mylist = list(dict.fromkeys(results))


                
                    else: 
                    
                                results.append('nocall')
                                
                                mylist = list(dict.fromkeys(results))
                                
                                        
                    if len(mylist)>1 and 'nocall' in mylist:
                        
                        mylist.remove('nocall')
                        prediction_dict[row_id] = ','.join(mylist)        
                    else:  
                        prediction_dict[row_id] = ','.join(mylist)        
prediction_dict

/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/cor

CPU times: user 25.9 s, sys: 2.84 s, total: 28.7 s
Wall time: 32 s


{'site_1_41e6fe6504a34bf6846938ba78d13df1_5': 'nocall',
 'site_1_41e6fe6504a34bf6846938ba78d13df1_10': 'nocall',
 'site_1_41e6fe6504a34bf6846938ba78d13df1_15': 'aldfly',
 'site_1_41e6fe6504a34bf6846938ba78d13df1_20': 'nocall',
 'site_1_41e6fe6504a34bf6846938ba78d13df1_25': 'aldfly',
 'site_1_cce64fffafed40f2b2f3d3413ec1c4c2_5': 'aldfly',
 'site_1_cce64fffafed40f2b2f3d3413ec1c4c2_10': 'nocall',
 'site_1_cce64fffafed40f2b2f3d3413ec1c4c2_15': 'aldfly',
 'site_1_cce64fffafed40f2b2f3d3413ec1c4c2_20': 'nocall',
 'site_1_cce64fffafed40f2b2f3d3413ec1c4c2_25': 'nocall',
 'site_1_cce64fffafed40f2b2f3d3413ec1c4c2_30': 'nocall',
 'site_1_cce64fffafed40f2b2f3d3413ec1c4c2_35': 'aldfly',
 'site_1_99af324c881246949408c0b1ae54271f_5': 'aldfly',
 'site_1_99af324c881246949408c0b1ae54271f_10': 'aldfly',
 'site_1_99af324c881246949408c0b1ae54271f_15': 'aldfly',
 'site_1_99af324c881246949408c0b1ae54271f_20': 'aldfly',
 'site_1_99af324c881246949408c0b1ae54271f_25': 'aldfly',
 'site_1_99af324c881246949408c0b1a

In [15]:
row_id = list(prediction_dict.keys())
birds = list(prediction_dict.values())

prediction_df = pd.DataFrame({
    "row_id": row_id,
    "birds": birds
})
prediction_df.to_csv("submission.csv", index=False)
